# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [36]:
import json
from collections import defaultdict

In [37]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [38]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [39]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [40]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

a. how many faculty taught COSI courses last year?

In [41]:
len({c['instructor'][2] for c in courses if c['subject']=='COSI'})

27

b. what is the total number of students taking COSI courses last year?

In [42]:
students=0
for c in courses:
    if c['subject']=='COSI':
        students+=c['enrolled']
students

2223

c. what was the median size of a COSI course last year (counting only those courses with at least 10 students)

In [43]:
course_by_size=sorted([c['enrolled'] for c in courses if c['enrolled']>=10])
res=0
if len(course_by_size)%2==1:
    res=course_by_size[len(course_by_size)//2]
else:
    res=course_by_size[len(course_by_size)//2-1]+course_by_size[len(course_by_size)//2+1]/2
res

30.0

d. create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.

In [47]:
d=defaultdict(int)
for c in courses:
    d[c['subject']]+=c['enrolled']
res=sorted([tup for tup in d.items()],key = lambda t:-t[1])
res[:10]

[('HS', 5318),
 ('BIOL', 3085),
 ('BUS', 2766),
 ('HWL', 2734),
 ('CHEM', 2322),
 ('ECON', 2315),
 ('COSI', 2223),
 ('MATH', 1785),
 ('PSYC', 1704),
 ('ANTH', 1144)]

e. do the same as in (d) but print the top 10 subjects in terms of number of courses offered

In [48]:
d=defaultdict(int)
for c in courses:
    d[c['subject']]+=1
res=sorted([tup for tup in d.items()],key = lambda t:-t[1])
res[:10]

[('BIOL', 613),
 ('HIST', 498),
 ('PSYC', 417),
 ('NEUR', 403),
 ('BCHM', 296),
 ('PHYS', 288),
 ('HS', 274),
 ('COSI', 272),
 ('MUS', 266),
 ('ENG', 265)]

f. do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject

In [49]:
d=defaultdict(set)
for c in courses:
    d[c['subject']].add(c['instructor'][2])
res=sorted([(tup[0],len(tup[1])) for tup in d.items()],key = lambda t:-t[1])
res[:10]

[('HS', 84),
 ('BIOL', 67),
 ('ECON', 52),
 ('BCHM', 49),
 ('HIST', 47),
 ('BUS', 46),
 ('BCBP', 46),
 ('HWL', 42),
 ('MATH', 37),
 ('NEJS', 37)]

i. own interesting question: 
what is the top 3 course with the longest course name?

In [50]:
res=sorted([c['name'] for c in courses],key = lambda t:-len(t))
res[:3]

['From Les Confessions to Instagram: Self-Writing in Contemporary French and Francophone Literature',
 'Chinese for Life Science: Study of Traditional Chinese Medicinal Herbs with Contemporary Science',
 'Seminar: Red States, Blue States: Understanding Contemporary American Voters and Parties']

In [ ]:
## all the subjects of courses

subjects = [course['subject'] for course in courses]
subjects[:5]

In [ ]:
# terms represented

{course['term'] for course in courses}

In [ ]:
#how many instructors taught at brandeis with at least 1 student last year

len({course['instructor'] for course in courses if course['enrolled'] > 0})

In [ ]:
# display subjects

{course['subject'] for course in courses}

In [ ]:
# hello

{course['name'] for course in courses}